In [5]:
from collections import defaultdict
import jsonlines
# get_gpt_generation() function from utils
from utils import * 

In [ ]:
# summarization
from src.summarization import *
import evaluate
import time

rouge = evaluate.load('rouge')

In [7]:
def summarization(test_examples, dataset_name):    
    gens, tgts, results, rouge_1 = [], [], [], []
    
    for idx, example in enumerate(test_examples):
        
        prompt = example['prompt'][:-1]
        tgt = example['tgt']

    ###### HERE YOU CAN CHANGE IT TO YOUR OWN MODELS########
        gen = get_gpt_generation(prompt)
    ########################################################
        gens.append(gen)
        tgts.append(tgt)
    
        obj={
            'gen': gen,
            'tgt': tgt}
    
        with jsonlines.open(f'results/summarization_result_{dataset_name}.jsonl', mode='a') as writer:
            writer.write(obj)

        if idx % 10 == 0:
            print(idx)
            time.sleep(5)
    

## CNNDM

In [8]:
test_examples, test_ids = CNNDM()
summarization(test_examples, 'cnndm')

0


## SAMSum

In [ ]:
test_examples, test_ids = SAMSum()
summarization(test_examples, 'samsum')

## Calculate Rouge

In [13]:
def get_rouge_score(output_file_dir):
    
    chatgpt_summary = []
    reference = []
    
    
    with jsonlines.open(output_file_dir) as read_file:
        for line in read_file.iter():
        	chatgpt_summary.append(line['gen'])
        	reference.append(line['tgt'])
            
    results = rouge.compute(predictions=chatgpt_summary,references=reference)
    
    return results['rouge1']

In [21]:
OUTPUT_FILES = ['results/summarization_result_cnndm.jsonl', 'results/summarization_result_samsum.jsonl']

for file in OUTPUT_FILES:
    rouge_score =  get_rouge_score(file)
    print(file, " :", round(rouge_score, 4))

results/summarization_result_cnndm.jsonl  : 0.323
results/summarization_result_samsum.jsonl  : 0.3326
